# PNN for Brain Tumor Detection based on GLCM

## Import the Dataset and Libraries
In this section we need to get the dataset. The dataset is available on a github repository. The repository contains images of brain tumor and normal brain images. The author has also performed augmentation on the dataset, and it is also provided. We use the augmented images for our training purpose.
<br>Once the repository is cloned, we will import all the required libraries that we need for our problem.

In [ ]:
!git  clone https://github.com/MohamedAliHabib/Brain-Tumor-Detection

In [ ]:
# Import necessary libraries
import numpy as np
import os
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt  # Import plt module for visualization
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from skimage.transform import resize
from skimage.feature import graycoprops, graycomatrix
from sklearn.model_selection import train_test_split

## Installing required libraries
We need to install the libraries we are going to use for defining an example PNN for our problem.

In [ ]:
!pip install neupy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of neupy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.2/197.2 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 42.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

In [ ]:
!pip install neurolab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for neurolab: filename=neurolab-0.3.5-py3-none-any.whl size=22180 sha256=f86d07f152e198705c88119b78d2273790055720f90d96326810ad906f56e879
  Stored in directory: /root/.cache/pip/wheels/1d/c0/44/7142fa43c89473c5e63a750a00224e5f9ec9ca80613de1f97d
Successfully built neurolab


## Declaring Variables

In [ ]:
# Set directory path and parameters
data_dir = '/content/Brain-Tumor-Detection/augmented data'
categories = ['yes', 'no']  # Names of the categories
img_size = 256  # Size of the image after resizing
num_classes = len(categories)
batch_size = 16

## Feature Extraction
Here will define a function named extract_features(). this method takes image path as input and returns the features for each image. Alongwith the feature extraction, the preprocessing on the images is done. <br>
In this method, we calculate the GLCM features based on contrast, dissimilarity, homogeneity, ASM,and energy.

In [ ]:
# Extract GLCM features
def extract_features(image_path):
    # Load image and resize
    img = plt.imread(image_path)
    img = resize(img, (img_size, img_size), anti_aliasing=True)
    
    # Convert to grayscale
    img_gray = rgb2gray(img)
    
    # Calculate GLCM
    glcm = graycomatrix((img_gray * 255).astype('uint8'), distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    
    # Extract GLCM properties
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    ASM = graycoprops(glcm, 'ASM')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    
    # Return features as an array
    features = np.array([contrast, dissimilarity, homogeneity, ASM, energy, correlation])
    return features


## Extracting Features and Preparing for Training
In this step, we load each image, and calculate the features using the method defined earlier. <br>
Finally, the features are split in train/val categories for training and testing.

In [ ]:
# Load data and extract features
X = []
y = []
for category in categories:
    path = os.path.join(data_dir, category)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        features = extract_features(img_path)
        X.append(features)
        y.append(categories.index(category))
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Definition
Here, an example PNN is defined for the explaination purposes. And, this PNN can further be improved for attaining more accuracy.

In [ ]:
# Importing the required library for our PNN definition
import neurolab as nl
# Define the PNN model architecture
input_size = X_train.shape[1]
output_size = 1 # Binary classification task
pnn = nl.net.newp(nl.tool.minmax(X_train), output_size)

## Model Training
In this step, we train the model. We run the training for 10 epochs total. The hyperparameters (batch_size and epochs) can be changed for attaining better results.

In [ ]:
# Train the PNN model
pnn.train(X_train, y_train.reshape(-1, output_size), epochs=50)

The maximum number of train epochs is reached


[390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5,
 390.5]

## Model Testing
Now the accuracy and loss is calculated for our model. Here, val dataset is used, we can devide our dataset into three categories, train, test, and val as well. 

In [ ]:
# Evaluate the PNN model on the val data
predictions = pnn.sim(X_val)
accuracy = (predictions == y_val.reshape(-1, output_size)).mean()
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.5181598062953995
